# Intel Advisor

Es una herramienta que puede ser aplicada a algoritmos escritos en C, C++, Fortran, OpenMP, SYCL, Python u OpenCL para garantizar que estos algoritmos estén siendo usados a su máximo potencial en procesadores modernos. Esto es particularmente importante si se trabaja con algoritmos codificados para **arquitecturas múltiples**, es decir que integran CPUs, GPUs, FPGAs, etc. SYCL usado para computación paralela aplicado a GPUs también contiene partes que se ejecutan en CPUs y por lo tanto representa un ejemplo de aplicación de Advisor. 


En una primera instancia vamos a instalar Advisor del mismo modo que instalamos VTune. Intel nos ofrece dos opciones de instalación. La primera es instalar como parte de [Intel oneAPI Base Toolkit](https://www.intel.com/content/www/us/en/developer/tools/oneapi/base-toolkit-download.html) o de forma individual [Intel Advisor](https://www.intel.com/content/www/us/en/developer/articles/tool/oneapi-standalone-components.html#advisor). 


## Advisor instalación

Para esta clase, vamos a instalar Advisor de forma individual, indicando el sistema operativo que tenemos y la opción offline para garantizar trabajar con Advisor aún cuando no tengamos conexión a internet:

![alt text](advisor_opt.png)

Guardamos y ejecutamos el ejecutable adecuado para nuestro sistema: 

![alt text](advisor_inst1.png)

Aceptamos la configuración recomendada de instalación:

![alt text](advisor_inst2.png)

Opcionalmente, podemos aceptar o no que Intel recoja información técnica de nuestro equipo y proceso de instalación. Dependiendo de lo que elijamos, procedemos a la instalación final. Adicionalmente, y al igual que con VTune, Advisor nos propone instalar Microsoft Visual Studio, pero esto no es requerido para nuestros propósitos y se puede regularizar posteriormente si es necesario:

![alt text](advisor_inst3.png)

La interface de Advisor se muestra de la siguiente manera: 

![alt text](advisor_inst4.png)

Y las opciones que ofrece incluyen análisis roofline, threading, vectorización, offload, etc.:

![alt text](roofline_A.png)
https://images.app.goo.gl/W1FCVYZkKzg7dp8x5

## Análisis Roofline

Nuestro código contiene diferentes secciones con diferente **intensidad aritmética** medida en _número de operaciones ejecutadas por data byte_ (leido de memoria). **Cuan rápido pueden ejecutarse cada una de estas operaciones** se mide en _operaciones por segundo_ (o en FLOP/s si se trabaja con operaciones de punto flotante). 

![alt_text](roofline_1.png)

En teoría, la intensidad aritmética es fija, es decir el número de operaciones ejecutadas por byte es fijo, pero la velocidad de ejecución en flops u operaciones por segundo puede controlarse dependiendo de la implementación del software, es decir cómo escribimos nuestro código y del hardware que usamos. El hardware tiene limitaciones propias de su tecnología, es decir el máximo número de operaciones que se pueden ejecutar por segundo. Algunas veces, la capacidad de memoria del hardware puede también limitar el número de operaciones por segundo.  

![alt_text](roofline_2.png)

La combinación de estas dos limitantes se denomina roofline o techo y representa la performancia máxima que nuestro hardware puede proveer.

![alt_text](roofline_3.png)

Advisor nos indica qué parte de nuestro código puede ser mejorado, es decir tiene potencial de incrementar su velocidad. Normalmente estas secciones del código son indicadas en color rojo, mientras que aquellas que secciones que no se usan son coloreadas de verde. Las secciones que ya se encuentran optimizadas y su mejora no traería mejoras generales al algoritmo son coloreadas en amarillo. Así mismo, Advisor nos indica diferentes rooflines para operaciones vectoriales o escalares, o diferentes límites dados por la memoria disponible. 

![alt_text](roofline_5.png)


## Offload Advisor

Sistemas heterogéneos incluyen un CPU denominado *host* y uno o más *aceleradores* que pueden ser GPUs, FPGA o AI.  

![alt_text](offload_1.png)

Todo nuestro código en teoría puede ser ejecutado en el *host*. Sin embargo, ciertas secciones de nuestro código pueden ejecutarse de manera más óptima en otros sistemas como GPUs, FPGAs, etc. Idealmente quisieramos saber:
i) qué partes de este código se ejecutan mejor en qué tipo de hardware, y 
ii) si vale la pena hacer ese cambio tomando en cuenta que la transferencia de datos del *host* al acelerador puede reducir su performancia, todo esto antes de adquirir algún hardware específico como un GPU. 

**Advisor, nos recomienda qué funciones de nuestro código pueden ser enviadas al GPU del CPU y proyecta la performancia de esas modificaciones.** 

Advisor analiza nuestro código y prepara una línea base de performancia usando nuestro hardware local. Luego, usando métodos heurísticos, analiza la performancia de cada parte del código en diferentes hardware como GPUs, FPGA, etc, usando para eso diferentes criterios como la capacidad de memoria, el costo de transferencia de datos, etc. (Nota: un método heurístico se usa para dar una solución satisfactoria a un problema en un tiempo rasonable pero no garantiza que esta solución sea la solución más óptima).

![alt_text](offload_2.png)

De este modo, Advisor offload sugiere qué partes del código pueden ser transferidos del CPU al GPU o de un GPU a otro de mejores capacidades técnicas. 

## Ejemplo de aplicación: Mandelbrot

Tomando la aplicación de la clase pasada, para poder utilizar Advisor, necesitamos un ejecutable de nuestro archivo escrito en SYCL. Para ello, localizamos la carpeta `mandelbrot/build/src` y ubicamos el ejecutable `mandelbrot`. La forma de ejecutar Advisor sigue la siguiente sintáxis:

`advisor --acción --opciones de acción --opciones globales --proceso deseado`

Utilizar la siguiente guía [cheet_sheet](https://www.intel.com/content/dam/develop/external/us/en/documents/advisor-cheat-sheet.pdf) para mayor información de procedimientos específicos.

1. Roofline: Una vez ubicada la carpeta, ejecutamos el siguiente comando:

`advisor --collect=roofline --project-dir=./advi_results -- /home/u183985/Compu_paralela_III/CPAR_GPUs/oneAPI-samples/DirectProgramming/C++SYCL/CombinationalLogic/mandelbrot/build/src/mandelbrot`

Dónde primero llamamos a la función `advisor`, se le señala la acción a ejecutar que es en este caso `collect=roofline`, luego creamos una carpeta denominada `advi_results` donde guardamos los resultados del roofline y finalmente le damos la ruta del ejecutable `mandelbrot`. Similarmente a Vtune, vamos a abrir los resultados en forma offline, para ello descargamos la carpeta `advi_results` usando los comandos `scp -r devcloud:/RUTA/NOMBRE de CARPETA .`
Antes de visualizar los resultados, no olvidar colocar en la misma carpeta el archivo fuente mandelbrot que no es generado en el análisis pero que es interesante para visualizar algunas sugerencias de Advisor. 

En la interface de Advisor, nos dirijimos a *Open Project* (abrir proyecto). El archivo de extensión *.advixeproj* es el que permite visualizar los resultados. Una vez abiertos, presionamos *show_results* ![alt_text](show_results.png). Los resultados nos muestra el techo o roofline del nodo en el que hemos realizado nuestro trabajo (GPU de Intel), pero hay la posibilidad de mostrar resultados supuestos en otro hardware. 

![alt_text](roofline_results.png)

Adicionalmente, roofline nos permite visualizar los hotspots en el archivo fuente muy similar a VTune:

![alt_text](roofline_results_1.png)

2. Offload: Una vez ubicada la carpeta, ejecutamos el siguiente comando:

`advisor --collect=offload --accuracy=low --project-dir=./advi_results_1 -- /home/u183985/Compu_paralela_III/CPAR_GPUs/oneAPI-samples/DirectProgramming/C++SYCL/CombinationalLogic/mandelbrot/build/src/mandelbrot`

Offload tiene diferentes niveles de precisión del análisis que van desde bajo (low), medio o estándar (medium) y alto (high). La diferencia de estas opciones está en el número de resultados que ofrecen y el tiempo de ejecución. Dado que nuestro ejemplo es relativamente sencillo vamos a usar el nivel de precisión bajo. Luego de ejecutado el comando, descargamos la carpeta `advi_results_1` para visualizarla offline. Sin embargo, ya la línea de comando nos da unos resultados interesantes cómo que hay dos regiones `Number of Offloads: 2` que pueden ser enviadas a un acelerador (GPU por ejemplo). Estas dos secciones representan el 15% de todo el código y tienen una potencialidad de aceleración de 16.2x:

![alt_text](offload_results.png)

Para ver estos resultados de manera más interactiva, descarguemos la carpeta a nuestra computadora local usando los comandos `scp -r devcloud:/RUTA/NOMBRE de CARPETA .` y abramos los resultados en Advisor:

![alt_text](offload_results_1.png)

Un resultado interesante ligado a esas dos regiones mencionadas anteriormente es que si si son enviadas a GPU, se podría mejorar la performancia del código total en 17%. Esto se puede visualizar en valor del Amdahl´s Law Speed up. Adicionalmente, en este ejemplo, el área **Non offloaded** es decir que no está en GPU es la que limita principalmente la performancia del código en un 85%. Finalmente, podemos observar algunas recomendaciones precisas de cómo mejorar el código que incluyen usar DPC++ parallel for o un pragma omp: 

![alt_text](offload_results_2.png)

# Referencias
- https://www.intel.com/content/www/us/en/developer/tools/oneapi/advisor.html#gs.qpvb2a
- https://www.youtube.com/watch?v=K4S5-q6ydi8
- https://www.youtube.com/watch?v=Clg4f9EI2vY 
- https://www.intel.com/content/www/us/en/develop/documentation/get-started-with-advisor/top.html
- https://www.intel.com/content/www/us/en/develop/documentation/advisor-user-guide/top/model-offloading-to-a-gpu.html
- https://www.intel.com/content/www/us/en/develop/documentation/advisor-user-guide/top/model-offloading-to-a-gpu/run-offload-modeling-perspective-from-command-line.html#run-offload-modeling-perspective-from-command-line
- https://www.intel.com/content/www/us/en/develop/documentation/advisor-user-guide/top/analyze-gpu-roofline.html
- https://www.intel.com/content/dam/develop/external/us/en/documents/advisor-cheat-sheet.pdf